In [1]:
import pandas as pd

# Load the trip data
trip_data = pd.read_csv('bicycle_trip_data.csv')

# Initial number of rows
num_rows_step0 = trip_data.shape[0]

# Convert 'Start date' to datetime
trip_data['Start date'] = pd.to_datetime(trip_data['Start date'])

# Only consider the last 3 months of the data
trip_data = trip_data[trip_data['Start date'] >= trip_data['Start date'].max() - pd.Timedelta('90 days')]
num_rows_step1 = trip_data.shape[0]
print(f"First date: {trip_data['Start date'].min()}")
print(f"Last date: {trip_data['Start date'].max()}")
print(f"This step reduced the number of rows from {num_rows_step0} to {num_rows_step1} ({1-(num_rows_step1/num_rows_step0):.2%})")
print(f"In Comparison to the original data we now look at {(num_rows_step1/num_rows_step0):.2%} of the data.")

# Only consider weekdays
trip_data['weekday'] = trip_data['Start date'].dt.weekday
trip_data = trip_data[trip_data['weekday'] < 5]
num_rows_step2 = trip_data.shape[0]
print(f"This step reduced the number of rows from {num_rows_step1} to {num_rows_step2} ({1-(num_rows_step2/num_rows_step1):.2%})")
print(f"In Comparison to the original data we now look at {(num_rows_step2/num_rows_step0):.2%} of the data.")

# Record the start and end station name and number for the filtered trips
station_data = trip_data[['Start station', 'Start station number', 'End station', 'End station number']]

# Save the station data to a new CSV file
station_data.to_csv('station_name_and_nbr.csv', index=False)
print("Station data saved to 'station_name_and_nbr.csv'")


/var/folders/zw/9vch0yrs50jfl_28l8p1__m80000gn/T/ipykernel_4075/3471039337.py:4: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  trip_data = pd.read_csv('bicycle_trip_data.csv')


First date: 2024-02-01 00:00:00
Last date: 2024-05-01 00:00:00
This step reduced the number of rows from 13514122 to 1896920 (85.96%)
In Comparison to the original data we now look at 14.04% of the data.
This step reduced the number of rows from 1896920 to 1443245 (23.92%)
In Comparison to the original data we now look at 10.68% of the data.
Station data saved to 'station_name_and_nbr.csv'


In [5]:
import pandas as pd

# Load the datasets
trip_data = pd.read_csv('cleaned_trip_data.csv')
station_data = pd.read_csv('station_name_and_nbr.csv')

# Split station data into two separate DataFrames for start and end stations
start_station_data = station_data[['Start station', 'Start station number']].drop_duplicates().rename(columns={'Start station': 'Station', 'Start station number': 'Station number'})
end_station_data = station_data[['End station', 'End station number']].drop_duplicates().rename(columns={'End station': 'Station', 'End station number': 'Station number'})

# Define a function to merge in smaller chunks incrementally
def incremental_merge(trip_data, start_station_data, end_station_data, chunk_size=50000):
    chunks = []
    for start in range(0, len(trip_data), chunk_size):
        trip_chunk = trip_data.iloc[start:start + chunk_size]
        trip_chunk = pd.merge(trip_chunk, start_station_data, left_on='Start station', right_on='Station', how='left').rename(columns={'Station number': 'Start station number'}).drop(columns=['Station'])
        trip_chunk = pd.merge(trip_chunk, end_station_data, left_on='End station', right_on='Station', how='left').rename(columns={'Station number': 'End station number'}).drop(columns=['Station'])
        chunks.append(trip_chunk)
    return pd.concat(chunks, ignore_index=True)

# Perform the incremental merge
trip_data_merged = incremental_merge(trip_data, start_station_data, end_station_data)

# Save the merged dataset to a CSV file
output_file_path = 'complete_data_final.csv'
trip_data_merged.to_csv(output_file_path, index=False)
